# Logistic Regression from Scratch

In [ ]:
# Import any required modules here
import numpy as np
import matplotlib.pyplot as plt


In this notebook we will be coding up Logistic Regression from scratch. We're doing this for two main reasons:
1. To make sure we understand the model fully
1. The step from this to neural networks is small

Once you've mastered coding up and this kind of model, and fitting its parameters with gradient descent, all the fancy neural network literature will be much more accessible.

## The sigmoid function

Firstly, lets get to grips with the sigmoid function. Recall: linear regression model took a D dimensional input and produced a single number, the prediction. This number could be any value on the real line. We can alter this function to restrict the output to be between 0 and 1 by using a sigmoid function. i.e. $f(x) = w_0*x_0 + w_1*x_1$ can become $f(x) = \sigma(w_0*x_0 + w_1*x_1)$.

#### Create a sigmoid function called `sig` using numpy and plot it with inputs ranging from -10 to 10

In [ ]:
def sig(x):
    output = 1/(1 + np.exp(-x))
    return output

xx = np.linspace(-10, 10, 100)
plt.plot(xx, sig(xx))
plt.xlabel('$x$')
plt.ylabel('$\sigma(x)$')
plt.grid(True)
plt.show()


If the 'linear regression equation' (i.e. the input to the sigmoid) returns a value <0, the output is <0.5. We could use a cutoff of 0.5 and return 'Class 0' for all inputs with an output of <0.5, and 'Class 1' for all inputs with an output of >=0.5. 

This is the inspiration for the Logistic Regression model: it returns a value between 0 and 1 which we can interpret as the probability of being in class 1.

## Creating the model

We're now going to create a function which performs the logistic regression on some dummy data. We're then going to visualise the result.

#### Create a function `logistic_regression`
which takes as input:
* the data `X` - an NxD dimensional matrix of the data (N datapoints)
* a vector `w` - a D dimensional vector of the parameters

and returns:
* `p` - an N dimensional output of predicted probabilities

(you can use your `sig` function defined above)

In [ ]:
def logistic_regression(X, w):
    a = X @ w
    p = sig(a)
    return p


#### Dummy data

Run the below cell to create some dummy data

In [ ]:
np.random.seed(1337)
nr_zero = 10
nr_one = 20
X = np.vstack(
    [np.random.standard_normal((nr_zero, 2)),
     np.random.standard_normal((nr_one, 2))*.5 + [2, 2]]
)
y = np.hstack([np.zeros(nr_zero), np.ones(nr_one)])
plt.scatter(X[y==0, 0], X[y==0, 1], marker='o', label='class 0')
plt.scatter(X[y==1, 0], X[y==1, 1], marker='x', label='class 1')
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.legend()
plt.show()

#### Plot the output of `logistic_regression` with varying `w`
Use the code given below to plot the probabilities predicted by different parameter settings `w`. Try:
* [0, 0] (what class would this predict everywhere?)
* [0, 1]
* [1, 0]
* [-1, 1]
* [1, 1]
* [1, 2]
* [1, 3]
* [10, 30]
* [.1, .3]

Comment on what you notice.

In [ ]:
def plot_logistic_probs(w, X, y):
    """
    Plots the probability surface for a 2d logistic regression
    with parameters w, with data X, y
    """
    plt.scatter(X[y==0, -2], X[y==0, -1], marker='o', 
                color='k', edgecolors='w', label='class 0')
    plt.scatter(X[y==1, -2], X[y==1, -1], marker='X', 
                color='k', edgecolors='w', label='class 1')
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    ax = plt.gca()
    x1_range = ax.get_xlim()
    x2_range = ax.get_ylim()
    xx1 = np.linspace(*x1_range, 100)
    xx2 = np.linspace(*x2_range, 100)
    xx1, xx2 = np.meshgrid(xx1, xx2)
    X = np.c_[xx1.ravel(), xx2.ravel()]
    if len(w) == 3:
        X = np.hstack([np.ones([X.shape[0], 1]), X])
        b, w_1, w_2 = w
    else:
        b, w_1, w_2 = np.hstack(([0], w))
    p = logistic_regression(X, w)
    plt.imshow(p.reshape(100, 100), vmin=0, vmax=1, 
               extent=x1_range+x2_range, origin='lower', cmap='bwr')
    plt.colorbar()
    xx1 = np.linspace(*x1_range, 100)
    m = -w_1/w_2
    c = -b/w_2 
    xx2 = m*xx1 + c
    mask = (xx2<x2_range[1]) & (xx2>x2_range[0])
    if sum(mask) > 0:
        plt.plot(xx1[mask], xx2[mask], '--', label='decision boundary')
    plt.legend(loc='best')
    
plot_logistic_probs([-3, 1, 1], X, y)

In [ ]:
# run plot_logistic_probs with various values of w
# plot_logistic_probs([0, 0], X, y)
# plot_logistic_probs([0, 1], X, y)
# plot_logistic_probs([1, 0], X, y)
# plot_logistic_probs([-1, 1], X, y)
# plot_logistic_probs([1, 1], X, y)
# plot_logistic_probs([1, 2], X, y)
# plot_logistic_probs([1, 3], X, y)
# plot_logistic_probs([10, 30], X, y)
plot_logistic_probs([.1, .3], X, y)
# plot_logistic_probs([-1.8, 1, 1.1], X, y)


In [ ]:
# What did you find? See solutions for comments
# * All the decision boundaries (i.e. the line with p=0.5) pass through the origin (0, 0)
#     * we have not coded in a bias term yet
#     * the plotting function is written such that you can provide a size 3 weight vector
#     * by eye, plot_logistic_probs([-1.8, 1, 1.1], X, y) looks a pretty good fit
# * The larger the parameters, the 'sharper' the boundary
# * The slope of the decision boundary is the negative ratio of the parameters w
#     * i.e. if you multiply w by a constant, the boundary is the same
#     * (aside) it's actually -w_1/w_2 (see function for the calculation)
#     * (aside) great math exercise to derive! Start with sigma(z) = 0.5


## Fitting the model

Now we have played with the model and (maybe) tried to fit the parameters `w` ourselves, let's fit `w` automatically. Much of this follows from the linear regression notebook: we're going to use gradient descent to minimise a loss function. This time, the loss function is the cross-entropy loss fuction.

#### Create a function `cross_entropy`
Which takes as input:
* a vector y of true labels (either 0 or 1)
* a vector y_pred of predicted probabilities (between 0 and 1)

And outputs:
* a single value loss

Cross-entropy loss L is defined as:
$$
\begin{align}
L &= -\frac{1}{N}\sum_{n=1}^N\log\left( p_n^{y_n} (1-p_n)^{(1-y_n)}\right) \\
p_n &= y_\text{pred} = \sigma(x_n w)
\end{align}
$$

Dividing by N is a convention - multiplying by a constant doesn't change the shape of the function, so doesn't matter a great deal.

Check it works by plotting the function when y=1 and varying y_pred between 0.1 and 0.9. What happens at y_pred=0?

In [ ]:
def cross_entropy(y, y_pred):
    preds = np.hstack([(1-y_pred), y_pred])
    loss = -1/y.shape[0] * np.sum(np.log(preds[y.astype(int)]))
    return loss

In [ ]:
# we provide the plotting code
y_pred = np.linspace(.1, .9, 100)
losses = np.zeros(y_pred.shape)
for ii in range(y_pred.shape[0]):
    losses[ii] = cross_entropy(np.array([1]), y_pred[ii])
plt.plot(y_pred, losses)
plt.xlabel('$y_{pred}$')
plt.ylabel('$L$')
plt.title('Cross-entropy loss for y=1 and $y_{pred}$')
plt.grid(True)
plt.show()

#### Fitting `w` using cross entropy

##### Step 1: Initialise
1. First initialise $w$ to any array of size $2$.

1. We have defined $X$ above

In [ ]:
w = ...

w = np.array([0., 0.])


##### Step 2: Predicted Values
Create a function that outputs the predicted values $\hat{y}$ given the current value of $w$.

Recall that $\hat{y} = \sigma(Xw)$

*Hint: you've already done this above!*

In [ ]:
def predicted_values(w, X):
    # calculate y_pred
    a = X @ w
    y_pred = sig(a)
    return y_pred

##### Step 3: Loss Function
Create a function that calulates the current loss of our predictions $\hat{y} = \sigma(Xw)$ using the cross-entropy loss.

*Hint:* you've done all the work for this above!

In [ ]:
def loss_function(w, X, y):
    # calculate y_pred using predicted_values
    y_pred = predicted_values(w, X)
    
    
    # calculate the cross-entropy loss from y and y_pred
    loss = cross_entropy(y, y_pred)
    
    return loss

##### Step 4: Gradient
Recall that the gradient of our loss function is (after some algebraic manipulation):

$$\nabla L(\boldsymbol{w})  \quad\!\!=\quad\!\! \frac{1}{N}\boldsymbol{X}^T (\hat{\boldsymbol{y}} - \boldsymbol{y})$$

Create a function that returns the gradient for a given value of $w$ (and $X$ and $y$).


In [ ]:
def gradient(w, X, y):
    ...
    y_pred = predicted_values(w, X)
    N = y.shape[0]
    grad = 1/N * X.T @ (y_pred - y)
    return grad

##### Step 5: Optimise!
Let's again use our trust SimpleGD function from Module 1, perform 10000 steps of gradient descent and plot the evolution of the function values returned. This is identical to Linear Regression from here - see if you can remember how to code it up without peeking!

We recommend using small step size to begin with: try $\gamma = 10^{-2}$.

Hint: if your loss increases towards $\infty$, try resetting $w$ and reducing the stepsize.

In [ ]:
# Gradient descent function
def simpleGD(w0, f, g, gamma, nr_steps):
    history = np.zeros(nr_steps+1)
    history[0] = f(w0)
    w = w0
    for ii in range(nr_steps):
        # this formulation amounts to writing w = w - stepsize*g(w)
        w -= gamma*g(w)
        history[ii+1] = f(w)
    return w, history

In [ ]:
# create versions of your loss and gradient functions that only require `w` as an input
# (hint: you might want to use a lambda function.)

loss_Xy = lambda w: loss_function(w, X, y)
gradient_Xy = lambda w: gradient(w, X, y)


In [ ]:
# Perform gradient descent on your loss function, 
# w, history = simpleGD(...
# w = np.zeros(2)    # in case step size is too large and loss explodes
w, history = simpleGD(w, loss_Xy, gradient_Xy, 1e-2, 10000)


##### Step 6: Optimisation Diagnostics
Plot the history of your loss over your iterates. 
* Does it decrease or increase? 
    * If increasing, *reset your $w$* and optimize again with a smaller step size.
    * If this doesn't fix the issue, you may have a bug in your code.
* How quickly does it decrease?
* Does it look like it's converging?
* [**Question**] How might you speed up the convergence?

Hint: be *very* careful when increasing the step size!

In [ ]:
plt.plot(history)
plt.xlabel('iteration')
plt.ylabel('loss')
plt.show()


##### Step 7: Plot the fit!

You've fitted a the logistic regression parameters `w` using gradient descent!!!

Why don't you treat yourself and plot the fit using `plot_logistic_probs()` from above

In [ ]:
w

In [ ]:
plot_logistic_probs(w, X, y)


## Bonus: Fit `w` with a bias term!

We can prepend a column of zeros to the data, much like for linear regression. The crossentropy loss gradient stays the same. Revisit the linear_regression notebook for hints.

#### Edit your code to fit `w` but include a bias as the first term
You will find that plot_logistic_probs will take a `w` of size 3: it expects the first term (i.e. `w[0]`) to be the bias.

In [ ]:
w = np.zeros(3)
X = np.hstack([np.ones((X.shape[0], 1)), X])
w, history = simpleGD(w, loss_Xy, gradient_Xy, 1e-2, 10000)
plt.plot(history)
plt.xlabel('iteration')
plt.ylabel('loss')
plt.show()
print(w)
plot_logistic_probs(w, X, y)
